[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/twMr7/Python-Tutorial/blob/master/13-Pandas_Data_Analysis.ipynb)

# 13. Pandas 資料分析

[Pandas](http://pandas.pydata.org/) 也是建構於 Numpy 之上，主要設計的定位是用來資料處理及分析。 與 Numpy 陣列不同的地方在，Pandas 表格式的資料容器 `DataFrame` 可以存放及操作異質資料型態，資料欄位可以有標籤，易於處理 missing data、類別資料、與時間序列資料，而且針對常用的資料儲存格式提供了相當廣泛的輸出入的支援。

一下教材內容節錄自 [Pandas 官方文件](http://pandas.pydata.org/pandas-docs/stable/index.html)。

+ [**13.1 Series 與 DataFrame 基本認識**](#basic-datatype)
+ [**13.2 漏失數據處理**](#missing-data)

### § 使用 `pandas` 套件

In [ ]:
import numpy as np
import pandas as pd

<a id="basic-datatype"></a>

## 13.1 `Series` 與 `DataFrame` 基本認識

Pandas 主要的資料結構是 `Series` 與 `DataFrame`。
+ `Series` - 一維，有標籤，同質性（homogeneously-typed）的資料結構。
+ `DataFrame` - 二維，有欄位標籤，欄位異質性（heterogeneously-typed ）的資料結構。

In [ ]:
# Creating a Series by passing a list of values, letting pandas create a default integer index
pd.Series([1, 3, 5, np.nan, 6, 8])

In [ ]:
# Creating a DataFrame by passing a NumPy array, with a datetime index and labeled columns
dates = pd.date_range('20190401', periods=6)
print(dates)

df = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
df

In [ ]:
# Creating a DataFrame by passing a dict of objects that can be converted to series-like.
# 注意： 單一值會自動 broadcast
df2 = pd.DataFrame({'A': 1.,
                    'B': pd.Timestamp('20130102'),
                    'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                    'D': np.array([3] * 4, dtype='int32'),
                    'E': pd.Categorical(["test", "train", "test", "train"]),
                    'F': 'foo'})
df2

### § 基本數據檢視


In [ ]:
# 檢視前幾筆
df.head(3)

In [ ]:
# 檢視後幾筆
df.tail(3)

In [ ]:
# 檢視記錄序號（標籤）
df.index

In [ ]:
# 檢視記錄欄位名稱（標籤）
df.columns

In [ ]:
# 資料都是數值的話，轉成 numpy 陣列非常快
df.to_numpy()

In [ ]:
# 異質欄位資料也可以轉成 numpy 陣列的話，但成本較高，通常在 numpy 也不會比較容易處理異質資料
df2.to_numpy()

In [ ]:
# 基本統計描述（column-wise）：平均值、標準差、最小值、第一四分位數、中位數、第二四分位數、最大值
df.describe()

In [ ]:
# Transpose
df.T

In [ ]:
# 根據某欄位排序
df.sort_values(by='B')

### §  資料內容選取

可以使用存取 Python 序列容器或 Numpy 陣列的序號（indexing）、片段（slicing）語法，但是建議使用針對 Pandas 資料結構最佳化過的 `at()`、`iat()`、`loc()`、和 `iloc()` 方法。

In [ ]:
# 選取單一欄位，返回一個 Series
df['A']

# df['A'] 語法等同於 df.A

In [ ]:
# 片段選取記錄序號列（row）
df[1:3]

In [ ]:
# 使用序號標籤選取片段
df['20190402':'20190404']

In [ ]:
# 使用 loc()，序號標籤選取
df.loc['2019-04-02']

In [ ]:
# 二維 indexing 及部分欄位選取
df.loc[:, ['A', 'B']]

In [ ]:
# 使用位置選取記錄序號列（row）
df.iloc[3]

In [ ]:
# 類似 numpy 的序號與片段語法
df.iloc[3:5, 0:2]

In [ ]:
# 單一內容值的選取可以用 at, iat
df.iat[1, 1]

In [ ]:
# Boolean 陣列選取，使用單一欄位
df[df.A > 0]

In [ ]:
# Boolean 陣列選取
df[df > 0]

In [ ]:
# 追加一個欄位資料
df['E'] = ['one', 'one', 'two', 'three', 'four', 'three']
df

In [ ]:
# 使用 isin 選取
df[df['E'].isin(['two', 'four'])]

### § 合併

In [ ]:
# 串接，預設 axis 0
pd.concat([df[1:3], df[4:6]])

In [ ]:
# 追加數據列
s3 = df.iloc[3]
s3.name = pd.to_datetime('2019-04-07')

df.append(s3)

<a id="missing-data"></a>

## 13.2 漏失數據處理

漏失數據（missing data，又稱 **NA**）在 Pandas 中主要使用 Numpy 的 `NaN`（Not a Number）形態表示，但也不排除使用 Python 的 `None`。 要注意的是，兩個 `NaN` 互相比較不會相等，但是 `None` 會相等，所以偵測漏失數據要使用 Pandas 提供的函式。

In [ ]:
# NaN 永遠不會等於 NaN
print('(NaN == NaN) is', np.nan == np.nan)
# None 等於 None
print('(None == None) is', None == None)

In [ ]:
# 刪除非數值的欄位 'E' 
df.drop(columns=['E'], inplace=True)

# 製造含 NaN 的數據
dfmiss = df[df > 0]
dfmiss

In [ ]:
# 偵測 NA 返回 Boolean，注意：不能使用 dfmiss == np.nan 這樣的比較
dfmiss.isna()

In [ ]:
# 針對 datetime 類型資料，Pandas 內部另外提供了 `NaT` 的資料類型來代表漏失的時間數據。
dfmiss['T'] = pd.Timestamp('20190417')
print(dfmiss)

dfmiss.iloc[[1, 3, 4], [4]] = np.nan
print(dfmiss)

In [ ]:
# 敘述統計函式，sum 把 NA 當 0，mean, cunsum 掠過
print('column A sum =', dfmiss['A'].sum())
print('column A mean =', dfmiss['A'].mean())
print('column A cumsum =\n', dfmiss['A'].cumsum())

In [ ]:
# 把有漏失任何欄位值的記錄丟掉，也可指定全部欄位沒有值才丟
dfmiss.dropna()

In [ ]:
# 丟掉很可惜，填補值來用
print('【填補前】：\n{}\n'.format(dfmiss))
print('【填補 0】：\n{}\n'.format(dfmiss.fillna(0)))
print('【後向填補】：\n{}\n'.format(dfmiss.fillna(method='bfill')))
print('【前向填補】：\n{}\n'.format(dfmiss.fillna(method='ffill')))

In [53]:
# 使用內插值填補，部份方法來自 scipy.interpolate 模組
print('【填補前】：\n{}\n'.format(dfmiss))
print('【linear 內插填補】：\n{}\n'.format(dfmiss.interpolate(method='linear')))
print('【pchip 內插填補】：\n{}\n'.format(dfmiss.interpolate(method='pchip')))
print('【krogh 內插填補】：\n{}\n'.format(dfmiss.interpolate(method='krogh')))

【填補前】：
                   A         B         C         D          T
2019-04-01  3.309681  0.195189       NaN  2.191791 2019-04-17
2019-04-02  0.091311       NaN  0.469101       NaN        NaT
2019-04-03       NaN  0.130222  0.798331  0.371961 2019-04-17
2019-04-04  3.134309  0.546685       NaN  0.132854        NaT
2019-04-05       NaN  0.206821       NaN  0.591907        NaT
2019-04-06  0.798170  0.899065       NaN       NaN 2019-04-17

【linear 內插填補】：
                   A         B         C         D          T
2019-04-01  3.309681  0.195189       NaN  2.191791 2019-04-17
2019-04-02  0.091311  0.162706  0.469101  1.281876        NaT
2019-04-03  1.612810  0.130222  0.798331  0.371961 2019-04-17
2019-04-04  3.134309  0.546685  0.798331  0.132854        NaT
2019-04-05  1.966240  0.206821  0.798331  0.591907        NaT
2019-04-06  0.798170  0.899065  0.798331  0.591907 2019-04-17

【pchip 內插填補】：
                   A         B         C         D          T
2019-04-01  3.309681  0.195189  